In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import SimpleITK as sitk
import cactas as C
import os

In [3]:
def load_seed(seed_image_path, cta_image):
    seed_image = sitk.ReadImage(seed_image_path)
    seed_indices = sitk.GetArrayFromImage(seed_image)
    seeds = list(zip(*seed_indices.nonzero()))
    seeds = [tuple(int(x) for x in seed) for seed in seeds]
    seeds = [(seed[2], seed[1], seed[0]) for seed in seeds]

    intensity_values = []
    for seed in seeds:
        # Extract a small region around the seed point
        region_radius = [1, 1, 1]  # Define the size of the region around the seed point
        region = sitk.RegionOfInterest(cta_image, region_radius, seed)
        region_array = sitk.GetArrayFromImage(region)

        # Collect all intensity values of the region
        intensity_values.extend(region_array.flatten())

    return seeds, intensity_values

In [4]:
def apply_window_level(cta_image, window_width, window_level):
    image_data = sitk.GetArrayFromImage(cta_image)
    
    lower_bound = window_level - (window_width / 2)
    upper_bound = window_level + (window_width / 2)
    
    windowed_image = np.clip(image_data, lower_bound, upper_bound)
    
    windowed_image_sitk = sitk.GetImageFromArray(windowed_image)
    windowed_image_sitk.CopyInformation(windowed_image_sitk)
    
    return windowed_image_sitk

In [5]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS/'
TS_PATH='/raid/mpsych/CACTAS/DATA/TS/'
output_dir='/raid/mpsych/CACTAS/DATA/CA9'

In [6]:
images, labels = C.Helper.load_data(DATAPATH)

In [7]:
for image in images:
    num = image.split(".")[0]

    # Construct the full path to the CTA image
    cta_image_path = os.path.join(DATAPATH, image)
    cta_image = sitk.ReadImage(cta_image_path)
    

    # Assume seed file names are related to the CTA image file name
    seed_image_path_left = os.path.join(TS_PATH + "/" + num, f"common_carotid_artery_left.nii.gz")
    seed_image_path_right = os.path.join(TS_PATH+ "/" + num, f"common_carotid_artery_right.nii.gz")
    seeds_left, intensities_left = load_seed(seed_image_path_left, cta_image)
    seeds_right, intensities_right = load_seed(seed_image_path_right, cta_image)

    # Combine seeds and intensity values from both sides
    seeds = seeds_left + seeds_right
    intensity_values = intensities_left + intensities_right

    intensity_values_array = np.array(intensity_values)  
    intensity_values_array = intensity_values_array[intensity_values_array >= 0]
    sorted_intensity_values = np.sort(intensity_values_array)
    
    intraluminal_HU = np.mean(sorted_intensity_values) 
    window_width = intraluminal_HU * 2.07
    window_level = intraluminal_HU * 0.72
    

    windowed_cta_array = apply_window_level(cta_image, window_width, window_level)
    
    #median_intensity = np.median(sorted_intensity_values)
    mean_intensity = np.mean(sorted_intensity_values)

    lower_threshold = float(mean_intensity)
    upper_threshold = lower_threshold + 100 
    
    
    # Use adjusted thresholds in the ConnectedThreshold function
    output_image = sitk.ConnectedThreshold(windowed_cta_array, seedList=seeds,
                                           lower=lower_threshold, upper=upper_threshold)

    # Process the output image as before
    output_image_array = sitk.GetArrayFromImage(output_image)
    output_image_array[:45] = np.zeros_like(output_image_array[:45])
    
    if output_image_array.shape[0] > 300:
        output_image_array[240:] = np.zeros_like(output_image_array[240:])
    else:
        output_image_array[120:] = np.zeros_like(output_image_array[120:])

    output_image_modified = sitk.GetImageFromArray(output_image_array)
    output_image_modified.CopyInformation(output_image)
    output_image_modified.SetSpacing(output_image.GetSpacing())
    output_image_modified.SetOrigin(output_image.GetOrigin())
    output_image_modified.SetDirection(output_image.GetDirection())

    # Save the output image with a unique name
    output_image_path = os.path.join(output_dir, f"{num}.ca.seg.nrrd")
    #print(output_image_path)
    sitk.WriteImage(output_image_modified, output_image_path)

    #print(str(num) + " " +str(lower_threshold) + " " + str(upper_threshold))

5 422.45471128885146 572.4547112888515
18 299.58407000625056 449.58407000625056
24 349.2687762350662 499.2687762350662
60 270.7151298918497 420.7151298918497
95 282.8760171082829 432.8760171082829
9 476.4696078431373 626.4696078431373
23 312.7357660280279 462.7357660280279
97 220.3449487554905 370.34494875549046
55 233.66691244239632 383.6669124423963
69 338.39297046619475 488.39297046619475
45 257.5567472009428 407.5567472009428
35 316.5302576830426 466.5302576830426
21 298.87066130800144 448.87066130800144
51 345.5252999265486 495.5252999265486
66 176.4699718146997 326.4699718146997
91 249.21977012773775 399.2197701277378
63 322.8833894791397 472.8833894791397
42 245.03435475161987 395.03435475161984
15 220.65430267062314 370.6543026706231
52 318.34811920631273 468.34811920631273
26 277.4764678471575 427.4764678471575
86 192.29434977578475 342.2943497757848
33 317.98300259566275 467.98300259566275
6 171.05752427184467 321.05752427184467
46 324.28317601332594 474.28317601332594
53 206